In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
import math
import random
from skimage.feature import hog
from random import uniform

In [3]:
mnist = pd.read_csv('cp_sample.csv', sep=';')
mnist

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
mnist_train, mnist_test = train_test_split(mnist, test_size = 0.3, random_state = 4)
mnist_valid, mnist_test = train_test_split(mnist_test, test_size = 0.5, random_state = 4)

In [4]:
from sklearn.ensemble import RandomForestClassifier

In [5]:
r_f = RandomForestClassifier(n_estimators=10) # this initializes a random forest with 10 trees
r_f.fit(mnist_train.iloc[:,1:],mnist_train.label) 
# here we fit this random forest by giving the features of all the
# examples of the training set (columns 1 to last) and their labels

RandomForestClassifier(n_estimators=10)

In [6]:
r_f.predict(mnist_train.iloc[0:2,1:])

array([8, 9])

In [7]:
r_f_score_train=r_f.score(mnist_train.iloc[:,1:mnist.shape[1]], mnist_train.label)
r_f_score_valid=r_f.score(mnist_valid.iloc[:,1:mnist.shape[1]], mnist_valid.label)

r_f_score_train
r_f_score_valid

0.7466666666666667

In [8]:
def forest(mnist_train,mnist_valid,Y,deb):
    result= pd.DataFrame()
    tree=[]
    score=[]
    for i in range (1,31):
        mnist_train.columns = mnist_train.columns.astype(str)
        mnist_valid.columns = mnist_valid.columns.astype(str)
        r_f = RandomForestClassifier(n_estimators=i) # this initializes a random forest
        tree.append(r_f.fit(mnist_train.iloc[:,deb:],mnist_train[Y]))
        r_f_score_valid=r_f.score(mnist_valid.iloc[:,deb:mnist.shape[1]], mnist_valid[Y])
        score.append(r_f_score_valid)

    result['score']= score
    result['tree']= tree
    best_index = result['score'].idxmax()
    best_forest = RandomForestClassifier(n_estimators=best_index)
    forest=best_forest.fit(mnist_train.iloc[:,deb:],mnist_train.label)
    return result,forest


result=forest(mnist_train,mnist_valid,'label',1)
result
    
    

(       score                                               tree
 0   0.506667  (DecisionTreeClassifier(max_features='sqrt', r...
 1   0.460000  (DecisionTreeClassifier(max_features='sqrt', r...
 2   0.646667  (DecisionTreeClassifier(max_features='sqrt', r...
 3   0.713333  (DecisionTreeClassifier(max_features='sqrt', r...
 4   0.660000  (DecisionTreeClassifier(max_features='sqrt', r...
 5   0.693333  (DecisionTreeClassifier(max_features='sqrt', r...
 6   0.686667  (DecisionTreeClassifier(max_features='sqrt', r...
 7   0.673333  (DecisionTreeClassifier(max_features='sqrt', r...
 8   0.713333  (DecisionTreeClassifier(max_features='sqrt', r...
 9   0.733333  (DecisionTreeClassifier(max_features='sqrt', r...
 10  0.766667  (DecisionTreeClassifier(max_features='sqrt', r...
 11  0.733333  (DecisionTreeClassifier(max_features='sqrt', r...
 12  0.726667  (DecisionTreeClassifier(max_features='sqrt', r...
 13  0.713333  (DecisionTreeClassifier(max_features='sqrt', r...
 14  0.806667  (DecisionT

In [9]:
best_forest_error=1-result[1].score(mnist_test.iloc[:,1:mnist.shape[1]], mnist_test.label)
best_forest_error

0.11333333333333329

In [10]:
def my_hog(row, ori, cell):
    return(pd.Series(hog(row.iloc[1:].to_numpy().reshape(28,28,1), orientations=ori, pixels_per_cell=(cell, cell), cells_per_block=(1,1),channel_axis=2)))

In [11]:
hog_train = mnist_train.apply(my_hog, axis=1, args=(8,14))
hog_valid= mnist_valid.apply(my_hog, axis=1, args=(8,14))
hog_test = mnist_test.apply(my_hog, axis=1, args=(8,14))
hog_train['label'] = mnist_train.label
hog_valid['label'] = mnist_valid.label
hog_test['label'] = mnist_test.label

In [12]:
hog_train

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,label
715,0.502681,0.0,0.502681,0.0,0.491869,0.0,0.502681,0.0,0.331770,0.0,...,0.0,0.500000,0.0,0.500000,0.0,0.500000,0.0,0.500000,0.0,8
920,0.699372,0.0,0.699372,0.0,0.147506,0.0,0.000000,0.0,0.500000,0.0,...,0.0,0.681478,0.0,0.681478,0.0,0.000000,0.0,0.266785,0.0,9
295,0.465444,0.0,0.618883,0.0,0.618883,0.0,0.131647,0.0,0.500000,0.0,...,0.0,0.307510,0.0,0.543607,0.0,0.552236,0.0,0.552236,0.0,2
83,0.517744,0.0,0.517744,0.0,0.517744,0.0,0.442518,0.0,0.568812,0.0,...,0.0,0.577350,0.0,0.577350,0.0,0.577350,0.0,0.000000,0.0,0
942,0.604642,0.0,0.604642,0.0,0.377247,0.0,0.355672,0.0,0.603838,0.0,...,0.0,0.577350,0.0,0.577350,0.0,0.577350,0.0,0.000000,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897,0.577350,0.0,0.577350,0.0,0.577350,0.0,0.000000,0.0,0.555912,0.0,...,0.0,0.572579,0.0,0.572579,0.0,0.572579,0.0,0.128290,0.0,0
709,0.000000,0.0,1.000000,0.0,0.000000,0.0,0.000000,0.0,0.615943,0.0,...,0.0,0.656532,0.0,0.754298,0.0,0.000000,0.0,0.000000,0.0,1
439,0.707107,0.0,0.707107,0.0,0.000000,0.0,0.000000,0.0,0.566947,0.0,...,0.0,0.558947,0.0,0.558947,0.0,0.558947,0.0,0.250470,0.0,0
174,0.500000,0.0,0.500000,0.0,0.500000,0.0,0.500000,0.0,0.197028,0.0,...,0.0,0.500000,0.0,0.500000,0.0,0.500000,0.0,0.500000,0.0,5


In [13]:
hog_test.columns = hog_test.columns.astype(str)
test=forest(hog_train, hog_valid, 'label', 0)[1]
test.score(hog_test.iloc[:, 0:hog_test.shape[1]], hog_test['label'])
print(1-test.score(hog_test.iloc[:, 0:hog_test.shape[1]], hog_test['label']))


0.08666666666666667
